In [1]:
#install necessary libraries
!pip install -q transformers accelerate sentencepiece

import os, torch, json
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer, T5ForConditionalGeneration, T5Tokenizer
import pandas as pd

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [ ]:
#load t5 fine tuned model
t5_model_dir = "ananya-bontala/t5-base-ft-summarization"
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_dir, use_safetensors=True).to(device)
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_dir)

In [3]:
#Load Multi Tasking DistillBert
class MultiTaskDistilBERT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.distilbert = DistilBertModel.from_pretrained(config["model_name"])
        hidden_size = self.distilbert.config.dim
        self.dropout = nn.Dropout(config["dropout"])

        # Category head
        self.category_pre_classifier = nn.Linear(hidden_size, hidden_size)
        self.category_classifier = nn.Linear(hidden_size, config["num_categories"])

        # Sentiment head
        self.sentiment_pre_classifier = nn.Linear(hidden_size, hidden_size)
        self.sentiment_classifier = nn.Linear(hidden_size, config["num_sentiments"])

        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state
        pooled_output = hidden_state[:, 0]  # CLS token
        x = self.dropout(pooled_output)

        # Category
        category = self.category_pre_classifier(x)
        category = self.relu(category)
        category = self.category_classifier(category)

        # Sentiment
        sentiment = self.sentiment_pre_classifier(x)
        sentiment = self.relu(sentiment)
        sentiment = self.sentiment_classifier(sentiment)

        return category, sentiment

# Load checkpoint
distilbert_path = "/content/drive/MyDrive/distillbert"
ckpt = torch.load(f"{distilbert_path}/best_model.pt", map_location=device)
custom_config = ckpt["config"]

distilbert_model = MultiTaskDistilBERT(custom_config)
distilbert_model.load_state_dict(ckpt["model_state_dict"])
distilbert_model.to(device)
distilbert_model.eval()

distilbert_tokenizer = DistilBertTokenizer.from_pretrained(custom_config["model_name"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
#Define label mappings
id2category = {0: "business", 1: "entertainment", 2: "politics", 3: "sports", 4: "tech"}
id2sentiment = {0: "positive", 1: "neutral", 2: "negative"}

In [5]:
#helper functions
def summarize(text, max_length=200):
    inputs = t5_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = t5_model.generate(**inputs, max_length=max_length)
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

def predict(text):
    inputs = distilbert_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        cat_logits, sent_logits = distilbert_model(**inputs)

    cat_id = torch.argmax(cat_logits, dim=1).item()
    sent_id = torch.argmax(sent_logits, dim=1).item()

    category_label = id2category[cat_id]
    sentiment_label = id2sentiment[sent_id]
    return category_label, sentiment_label

def process_text(text):
    summary = summarize(text)
    category, sentiment = predict(text)
    return {
        "input": text,
        "summary": summary,
        "category": category,
        "sentiment": sentiment
    }



In [10]:
# Test
texts = ["""soul sensation ready for awards south west teenage singing sensation, joss stone, has been nominated in three categories in wednesday's brit awards. the 17-year-old from a small east devon village near cullumpton, received nominations for best solo female, best urban act and best breakthrough artist. her second album mind, body soul reached number one in the uk charts last october and went straight into the us charts at number 11. ms stone is due to perform at the 25th award ceremony at london's earls court. the teenager also has grammy nominations in the us, normally dominated by home-grown acts. born jocelyn stoker, the devon diva started her career in a bbc talent programme, and was then discovered at a new york audition by a us record executive, steve greenberg. the 17-year-old singer is hoping to tour in japan, australia and the us in 2005."""]
outputs = [process_text(t) for t in texts]

df = pd.DataFrame(outputs)
pd.set_option('display.max_colwidth', None)
df



,input,summary,category,sentiment
0,"soul sensation ready for awards south west teenage singing sensation, joss stone, has been nominated in three categories in wednesday's brit awards. the 17-year-old from a small east devon village near cullumpton, received nominations for best solo female, best urban act and best breakthrough artist. her second album mind, body soul reached number one in the uk charts last october and went straight into the us charts at number 11. ms stone is due to perform at the 25th award ceremony at london's earls court. the teenager also has grammy nominations in the us, normally dominated by home-grown acts. born jocelyn stoker, the devon diva started her career in a bbc talent programme, and was then discovered at a new york audition by a us record executive, steve greenberg. the 17-year-old singer is hoping to tour in japan, australia and the us in 2005.","soul sensation ready for awards south west teenage singing sensation, joss stone, has been nominated in three categories in wednesday's brit awards.the teenager also has grammy nominations in the us, normally dominated by home-grown acts.the 17-year-old singer is hoping to tour in japan, australia and the us in 2005.born jocelyn stoker, the devon diva started her career in a bbc talent programme, and was then discovered at a new york audition by a us record executive, steve greenberg.soutwest teenage singing sensation, joss stone, has been nominated in three categories in wednesday's brit awards.born jocelyn stoker, the devon diva started her career in a bbc talent programme, and was then discovered at a new",entertainment,positive


In [11]:
# Test
texts = ["""home phones face unclear future the fixed line phone in your home could soon be an endangered species. research by handset maker nokia shows that more and more people are using their mobile phone for every call they make or take. according to the study, more than 45 million people in the uk, germany, us and south korea now only use a mobile. it showed that people keep their fixed line phone because call charges are lower, but most of those questioned said the future was definitely mobile. the nokia-sponsored research showed that mobiles and fixed phones were used for different purposes. home phones were used for longer calls but conversations on mobiles tended to be shorter, between mobiles and to friends. in the uk 69 of those questioned said they turned to their fixed phone because it was still cheaper to use than a mobile. however, when pressed few could say with accuracy how tariffs on fixed and mobile phones compared. in the us and germany many of those interviewed said they used the fixed phone because it was more reliable than a mobile handset and let them get access to the net at relatively high speeds. in all the countries where interviews were carried out, older people were more likely to use a fixed line phone more than a mobile. women aged 50 or above almost never use a mobile phone, the research found. the move to mobile was most pronounced in south korea where 65 of those questioned said they already make most of their calls from a mobile. 18 said they would not get a landline if they moved house. many of those questioned said they had an emotional connection to their fixed phone that drew on its position in the home and the "cosiness" of making a call there. nokia said these findings had implications for mobile operators who must work hard to ensure that mobiles are seen as cheap, reliable and providing good call quality. the survey also showed that it is not just voice calls that are going wireless. some of those questioned said they were looking to use a mobile or wireless service to get net access within the next couple of years. polling firm mori interviewed more than 6,000 people in the uk, us, germany and south korea for the survey."""]
outputs = [process_text(t) for t in texts]

df = pd.DataFrame(outputs)
pd.set_option('display.max_colwidth', None)
df



,input,summary,category,sentiment
0,"home phones face unclear future the fixed line phone in your home could soon be an endangered species. research by handset maker nokia shows that more and more people are using their mobile phone for every call they make or take. according to the study, more than 45 million people in the uk, germany, us and south korea now only use a mobile. it showed that people keep their fixed line phone because call charges are lower, but most of those questioned said the future was definitely mobile. the nokia-sponsored research showed that mobiles and fixed phones were used for different purposes. home phones were used for longer calls but conversations on mobiles tended to be shorter, between mobiles and to friends. in the uk 69 of those questioned said they turned to their fixed phone because it was still cheaper to use than a mobile. however, when pressed few could say with accuracy how tariffs on fixed and mobile phones compared. in the us and germany many of those interviewed said they used the fixed phone because it was more reliable than a mobile handset and let them get access to the net at relatively high speeds. in all the countries where interviews were carried out, older people were more likely to use a fixed line phone more than a mobile. women aged 50 or above almost never use a mobile phone, the research found. the move to mobile was most pronounced in south korea where 65 of those questioned said they already make most of their calls from a mobile. 18 said they would not get a landline if they moved house. many of those questioned said they had an emotional connection to their fixed phone that drew on its position in the home and the ""cosiness"" of making a call there. nokia said these findings had implications for mobile operators who must work hard to ensure that mobiles are seen as cheap, reliable and providing good call quality. the survey also showed that it is not just voice calls that are going wireless. some of those questioned said they were looking to use a mobile or wireless service to get net access within the next couple of years. polling firm mori interviewed more than 6,000 people in the uk, us, germany and south korea for the survey.","in the uk 69 of those questioned said they turned to their fixed phone because it was still cheaper to use than a mobile.research by handset maker nokia shows that more and more people are using their mobile phone for every call they make or take.in the us and germany many of those interviewed said they used the fixed phone because it was more reliable than a mobile handset and let them get access to the net at relatively high speeds.in the uk 69 of those questioned said they turned to their fixed phone because it was still cheaper to use than a mobile.the nokia-sponsored research showed that mobiles and fixed phones were used for different purposes.it showed that people keep their fixed line phone because call charges are lower, but most of those questioned said the future was definitely mobile.home phones were used for longer calls but conversations on mobiles tended to be shorter, between mobiles and to friends.in the",tech,negative


In [12]:
# Test
texts = ["""saab to build cadillacs in sweden general motors, the world's largest car maker, has confirmed that it will build a new medium-sized cadillac bls at its loss-making saab factory in sweden. the car, unveiled at the geneva motor show, is intended to compete in the medium-sized luxury car market. it will not be sold in the us, said gm europe president carl-peter forster. as part of its efforts to make the us marque appeal to european drivers, the car will be the first cadillac with a diesel engine. gm's announcement should go some way to allay fears of the saab factory's closure. the factory in trollhaettan has been at the centre of rumours about gm's planned severe cutbacks in its troubled european operations. but the group's new commitment to the swedish factory may not be welcomed by the group's opel workers in ruesselsheim, germany. they may now have to face a larger proportion of gm's cuts. neither will the announcement be seen as unalloyed good news in sweden, since it reflects saab's failure to make significant inroads into the lucrative european luxury car market. for years, saab has consistently said it is competing head-on with bmw, mercedes and jaguar. the segment's leaders do not agree. gm's plans to build the american marque in sweden is part of its efforts to push it as an alternative luxury brand for european drivers. in the us, it has long been established as an upmarket brand - even the presidential limousine carries the badge. yet it could prove tough for cadillac to steal market share from the majors in europe. other luxury car makers, most notably the toyota subsidiary lexus, have enjoyed tremendous success in the us without managing to make significant inroads in europe. there, german marques mercedes benz and bmw have retained their stranglehold on the luxury market. bringing cadillac production to sweden should help introduce desperately-needed scale to the saab factory, which currently produces fewer than 130,000 cars per year. that is about half of what major car makers consider sufficient numbers for profitable operations, and saab is losing money fast - albeit with losses halved in 2004 to 200m ( 104m; 151m euros) from 500m the previous year. beyond the 12,000 job cuts announced last year at its european operations, gm is reducing expenditure by building saabs, opels - badged as vauxhalls in the uk - and now cadillacs on the same framework, and by allowing the different brands to share parts. another way to further reduce saab's losses could be to shift some of the production of saabs to the us, a market where drivers have adopted it as an upmarket european car. doing so would remove the exposure to the weak us dollar, which is making saabs more expensive to us consumers. but not everyone in the industry agree that it would be the best way forward. "we know that in five years the us dollar will be stronger than it is today," the chief executive of a leading european car maker told bbc news. the current trend towards us production was "stupid", he said. in a separate announcement, gm unveiled a new scheme to allow european consumers the chance to test drive its opel and vauxhall models. it is to deploy a fleet of 35,000 test cars across 40 countries, inviting potential buyers to try out a vehicle for 24-hours. it follows a similar initiative by gm in the us. gm said it wanted to change "customers' perceptions" about opel and vauxhall cars, showing them that the quality had improved in recent years."""]
outputs = [process_text(t) for t in texts]

df = pd.DataFrame(outputs)
pd.set_option('display.max_colwidth', None)
df



,input,summary,category,sentiment
0,"saab to build cadillacs in sweden general motors, the world's largest car maker, has confirmed that it will build a new medium-sized cadillac bls at its loss-making saab factory in sweden. the car, unveiled at the geneva motor show, is intended to compete in the medium-sized luxury car market. it will not be sold in the us, said gm europe president carl-peter forster. as part of its efforts to make the us marque appeal to european drivers, the car will be the first cadillac with a diesel engine. gm's announcement should go some way to allay fears of the saab factory's closure. the factory in trollhaettan has been at the centre of rumours about gm's planned severe cutbacks in its troubled european operations. but the group's new commitment to the swedish factory may not be welcomed by the group's opel workers in ruesselsheim, germany. they may now have to face a larger proportion of gm's cuts. neither will the announcement be seen as unalloyed good news in sweden, since it reflects saab's failure to make significant inroads into the lucrative european luxury car market. for years, saab has consistently said it is competing head-on with bmw, mercedes and jaguar. the segment's leaders do not agree. gm's plans to build the american marque in sweden is part of its efforts to push it as an alternative luxury brand for european drivers. in the us, it has long been established as an upmarket brand - even the presidential limousine carries the badge. yet it could prove tough for cadillac to steal market share from the majors in europe. other luxury car makers, most notably the toyota subsidiary lexus, have enjoyed tremendous success in the us without managing to make significant inroads in europe. there, german marques mercedes benz and bmw have retained their stranglehold on the luxury market. bringing cadillac production to sweden should help introduce desperately-needed scale to the saab factory, which currently produces fewer than 130,000 cars per year. that is about half of what major car makers consider sufficient numbers for profitable operations, and saab is losing money fast - albeit with losses halved in 2004 to 200m ( 104m; 151m euros) from 500m the previous year. beyond the 12,000 job cuts announced last year at its european operations, gm is reducing expenditure by building saabs, opels - badged as vauxhalls in the uk - and now cadillacs on the same framework, and by allowing the different brands to share parts. another way to further reduce saab's losses could be to shift some of the production of saabs to the us, a market where drivers have adopted it as an upmarket european car. doing so would remove the exposure to the weak us dollar, which is making saabs more expensive to us consumers. but not everyone in the industry agree that it would be the best way forward. ""we know that in five years the us dollar will be stronger than it is today,"" the chief executive of a leading european car maker told bbc news. the current trend towards us production was ""stupid"", he said. in a separate announcement, gm unveiled a new scheme to allow european consumers the chance to test drive its opel and vauxhall models. it is to deploy a fleet of 35,000 test cars across 40 countries, inviting potential buyers to try out a vehicle for 24-hours. it follows a similar initiative by gm in the us. gm said it wanted to change ""customers' perceptions"" about opel and vauxhall cars, showing them that the quality had improved in recent years.","saab to build cadillacs in sweden general motors, the world's largest car maker, has confirmed that it will build a new medium-sized cadillac bls at its loss-making saab factory in sweden.gm's plans to build the american marque in sweden is part of its efforts to push it as an alternative luxury brand for european drivers.gm's plans to build the american marque in sweden is part of its efforts to push it as an alternative luxury brand for european drivers.neither wi

In [13]:
# Test
texts = ["""jack cunningham to stand down veteran labour mp and former cabinet minister jack cunningham has said he will stand down at the next election. one of the few blair-era ministers to serve under jim callaghan, he was given the agriculture portfolio when labour regained power in 1997. mr cunningham went on to become tony blair's "cabinet enforcer". he has represented the constituency now known as copeland since 1970. mr blair said he was a "huge figure" in labour and a "valued, personal friend". during labour's long period in opposition, mr cunningham held a number of shadow roles including foreign affairs, the environment and as trade spokesman. as agriculture minister he caused controversy when he decided to ban beef on the bone in the wake of fears over bse. he quit the government in 1999 and in recent years has served as the chairman of the all-party committee on lords reform and has been a loyal supporter of the government from the backbenches."""]
outputs = [process_text(t) for t in texts]

df = pd.DataFrame(outputs)
pd.set_option('display.max_colwidth', None)
df



,input,summary,category,sentiment
0,"jack cunningham to stand down veteran labour mp and former cabinet minister jack cunningham has said he will stand down at the next election. one of the few blair-era ministers to serve under jim callaghan, he was given the agriculture portfolio when labour regained power in 1997. mr cunningham went on to become tony blair's ""cabinet enforcer"". he has represented the constituency now known as copeland since 1970. mr blair said he was a ""huge figure"" in labour and a ""valued, personal friend"". during labour's long period in opposition, mr cunningham held a number of shadow roles including foreign affairs, the environment and as trade spokesman. as agriculture minister he caused controversy when he decided to ban beef on the bone in the wake of fears over bse. he quit the government in 1999 and in recent years has served as the chairman of the all-party committee on lords reform and has been a loyal supporter of the government from the backbenches.","veteran labour mp and former cabinet minister jack cunningham has said he will stand down at the next election.mr blair said he was a ""huge figure"" in labour and a ""valued, personal friend"".mr cunningham went on to become tony blair's ""cabinet enforcer"".mr cunningham said he was a ""huge figure"" in labour and a ""valued, personal friend"".one of the few blair-era ministers to serve under jim callaghan, he was given the agriculture portfolio when labour regained power in 1997.veteran labour mp and former cabinet minister jack cunningham has said he will stand down at the next election.mr cunningham went on to become tony blair's ""cabinet enforcer"".m",politics,neutral


In [14]:
# Test
texts = ["""henman hopes ended in dubai third seed tim henman slumped to a straight sets defeat in his rain-interrupted dubai open quarter-final against ivan ljubicic. the croatian eighth seed booked his place in the last four with a 7-5 6-4 victory over the british number one. henman had looked on course to level the match after going 2-0 up in the second set, but his progress was halted as the rain intervened again. ljubicic hit back after the break to seal a fourth straight win over henman. earlier in the day, spanish fifth seed tommy robredo secured his semi-final place when he beat nicolas kiefer of germany 6-4 6-4. afterwards, henman was left cursing the weather and the umpire after seven breaks for rain during the match. "it was incredibly frustrating," henman said. "it's raining and the umpire doesn't take control. "he kept telling us to play till the end of the game. but if it's raining, you come off - the score's irrelevant. "it couldn't be more frustrating as i was very happy with my form until now. you don't expect this in the desert."""]
outputs = [process_text(t) for t in texts]

df = pd.DataFrame(outputs)
pd.set_option('display.max_colwidth', None)
df



,input,summary,category,sentiment
0,"henman hopes ended in dubai third seed tim henman slumped to a straight sets defeat in his rain-interrupted dubai open quarter-final against ivan ljubicic. the croatian eighth seed booked his place in the last four with a 7-5 6-4 victory over the british number one. henman had looked on course to level the match after going 2-0 up in the second set, but his progress was halted as the rain intervened again. ljubicic hit back after the break to seal a fourth straight win over henman. earlier in the day, spanish fifth seed tommy robredo secured his semi-final place when he beat nicolas kiefer of germany 6-4 6-4. afterwards, henman was left cursing the weather and the umpire after seven breaks for rain during the match. ""it was incredibly frustrating,"" henman said. ""it's raining and the umpire doesn't take control. ""he kept telling us to play till the end of the game. but if it's raining, you come off - the score's irrelevant. ""it couldn't be more frustrating as i was very happy with my form until now. you don't expect this in the desert.","afterwards, henman was left cursing the weather and the umpire after seven breaks for rain during the match.henman had looked on course to level the match after going 2-0 up in the second set, but his progress was halted as the rain intervened again.henman hopes ended in dubai third seed tim henman slumped to a straight sets defeat in his rain-interrupted dubai open quarter-final against ivan ljubicic.henman had looked on course to level the match after going 2-0 up in the second set, but his progress was halted as the rain intervened again.it couldn't be more frustrating as i was very happy with my form until now.it was incredibly frustrating,"" henman said.it couldn't be more frustrating as i was very happy with my form until now.ljubicic hit back after the",sports,negative
